# 作業

實作各種不一樣的方式來調整飽和 / 對比 / 明亮

1. 改變 color space 來調整飽和度
2. 實作直方圖均衡
3. alpha/ beta 調整對比 / 明亮

In [1]:
import cv2
import numpy as np

In [2]:
img_path = 'D:/Tom/GitHub/DL_CVMarathon/img/Day_001/lena.png'

# 以彩色圖片的方式載入
img_bgr = cv2.imread( img_path, cv2.IMREAD_COLOR )

## 改變飽和度

- 轉換成 HSV color space, 改變 s channel 的值

In [3]:
# 為了要改變飽和度，我們先把 color space 轉成 HSV 格式
img_hsv = cv2.cvtColor( img_bgr, cv2.COLOR_BGR2HSV )

# 設定調降或增加飽和度的比例( 以兩成為例 )
change_percentage = 0.2

In [4]:
''' 在 HSV color space 減少飽和度 '''
# 針對飽和度的值做改變，超過界線 0~1 的都會 bound

img_hsv_down = img_hsv.astype( 'float32' )
img_hsv_down[ ..., 1 ] = img_hsv_down[ ..., 1 ] / 255 - change_percentage
img_hsv_down[ img_hsv_down[..., 1] < 0 ] = 0
img_hsv_down[ ..., 1 ] = img_hsv_down[ ..., 1 ] * 255
img_hsv_down = img_hsv_down.astype( 'uint8' )
img_bgr_down = cv2.cvtColor( img_hsv_down, cv2.COLOR_HSV2BGR )  # 轉換 Color Space 回 BGR

In [5]:
''' 在 HSV color space 增加飽和度 '''
# 針對飽和度的值做改變，超過界線 0~1 的都會 bound

img_hsv_up = img_hsv.astype( 'float32' )
img_hsv_up[ ..., 1 ] = img_hsv_up[ ..., 1 ] / 255 + change_percentage
img_hsv_up[ img_hsv_up[..., 1] > 1 ] = 1
img_hsv_up[ ..., 1 ] = img_hsv_up[ ..., 1 ] * 255
img_hsv_up = img_hsv_up.astype( 'uint8' )
img_bgr_up = cv2.cvtColor( img_hsv_up, cv2.COLOR_HSV2BGR )  # 轉換 Color Space 回 BGR

In [6]:
# 組合圖片 + 顯示圖片
img_hsv_change = np.hstack( ( img_bgr, img_bgr_down, img_bgr_up ) )

while True:
    cv2.imshow( 'Change Saturation', img_hsv_change )
    cv2.imwrite( 'Change_Saturation_by_Color_Space.jpg', img_hsv_change )
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 直方圖均衡

* case 1: 把彩圖拆開對每個 channel 個別做直方圖均衡再組合起來
* case 2: 轉換 color space 到 HSV 之後對其中一個 channel 做直方圖均衡

In [7]:
''' Case 1 '''
# 每個 channel 個別做直方圖均衡
equalHist_by_channel = [ img_bgr[ ..., 0 ], img_bgr[ ..., 1 ], img_bgr[ ..., 2 ] ]
equalHist_by_channel = [ cv2.equalizeHist(i) for i in equalHist_by_channel ]

# 組合經過直方圖均衡的每個 channel
img_bgr_equal = np.stack( equalHist_by_channel, axis = -1 )

In [8]:
''' Case 2 '''
# 轉換 Color Space 後只對其中一個 channel 做直方圖均衡
img_hsv = cv2.cvtColor( img_bgr, cv2.COLOR_BGR2HSV ) 
img_hsv[ ..., -1 ] = cv2.equalizeHist( img_hsv[ ..., -1 ] )
img_hsv_equal = cv2.cvtColor( img_hsv, cv2.COLOR_HSV2BGR )

In [9]:
# 組合圖片 + 顯示圖片
img_bgr_equalHist = np.hstack((img_bgr, img_bgr_equal, img_hsv_equal))
while True:
    # 比較 (原圖, BGR color space 對每個 channel 做直方圖均衡, HSV color space 對明度做直方圖均衡)
    cv2.imshow( 'bgr equal histogram', img_bgr_equalHist)
    cv2.imwrite( 'Change_Value.jpg', img_bgr_equalHist )
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break

## 調整對比 / 明亮

In [10]:
# alpha: 控制對比度 (1.0~3.0)
# beta: 控制明亮度 (0~255)
add_contrast = cv2.convertScaleAbs( img_bgr, alpha = 2.0, beta = 0 )
add_lighness = cv2.convertScaleAbs( img_bgr, alpha = 0.5, beta = 150 )

# 組合圖片 + 顯示圖片
img_contrast_light = np.hstack((img_bgr, add_contrast, add_lighness))
while True:
    # 比較不同程度的對比 / 明亮
    cv2.imshow( 'Adjust Contrast and Brighness', img_contrast_light )
    cv2.imwrite( 'Adjust_Contrast_and_Brighness.jpg', img_contrast_light )
    
    k = cv2.waitKey(0)
    if k == 27:
        cv2.destroyAllWindows()
        break